In [2]:
import hopsworks
import pandas as pd

In [3]:
project = hopsworks.login()
fs = project.get_feature_store()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/398
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
titanic_passengers_df = pd.read_csv("https://repo.hops.works/dev/jdowling/titanic/titanic_passengers.csv")
titanic_passengers_df

,passenger_id,name,sex,age
0,1,"Braund, Mr. Owen Harris",male,22.0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0
2,3,"Heikkinen, Miss. Laina",female,26.0
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0
4,5,"Allen, Mr. William Henry",male,35.0
...,...,...,...,...
886,887,"Montvila, Rev. Juozas",male,27.0
887,888,"Graham, Miss. Margaret Edith",female,19.0
888,889,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0
889,890,"Behr, Mr. Karl Howell",male,26.0


In [5]:
titanic_tickets_df = pd.read_csv("https://repo.hops.works/dev/jdowling/titanic/titanic_tickets.csv")
titanic_tickets_df

,passenger_id,fare,embarked
0,1,7.2500,S
1,2,71.2833,C
2,3,7.9250,S
3,4,53.1000,S
4,5,8.0500,S
...,...,...,...
886,887,13.0000,S
887,888,30.0000,S
888,889,23.4500,S
889,890,30.0000,C


In [6]:
titanic_survived_df = pd.read_csv("https://repo.hops.works/dev/jdowling/titanic/titanic_survived.csv")
titanic_survived_df

,passenger_id,survived
0,1,0
1,2,1
2,3,1
3,4,1
4,5,0
...,...,...
886,887,0
887,888,1
888,889,0
889,890,1


## Feature Engineering

Encode all the categorical features as number

In [7]:
titanic_passengers_df['sex'].replace('female', 0,inplace=True)
titanic_passengers_df['sex'].replace('male', 1,inplace=True)
titanic_passengers_df['sex'].astype(int)


titanic_passengers_df['age_bin'] = pd.cut(titanic_passengers_df['age'], bins=[0,12,19,40, 65, 130], 
                                          labels=['child','teen','young_adult','middle_aged','pensioner'])
titanic_passengers_df = pd.get_dummies(titanic_passengers_df, columns=["age_bin"], prefix=["age"])

titanic_passengers_df

,passenger_id,name,sex,age,age_child,age_teen,age_young_adult,age_middle_aged,age_pensioner
0,1,"Braund, Mr. Owen Harris",1,22.0,0,0,1,0,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,0,0,1,0,0
2,3,"Heikkinen, Miss. Laina",0,26.0,0,0,1,0,0
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,0,0,1,0,0
4,5,"Allen, Mr. William Henry",1,35.0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
886,887,"Montvila, Rev. Juozas",1,27.0,0,0,1,0,0
887,888,"Graham, Miss. Margaret Edith",0,19.0,0,1,0,0,0
888,889,"Johnston, Miss. Catherine Helen ""Carrie""",0,28.0,0,0,1,0,0
889,890,"Behr, Mr. Karl Howell",1,26.0,0,0,1,0,0


In [8]:
titanic_tickets_df['embarked'].replace('S', 0,inplace=True)
titanic_tickets_df['embarked'].replace('C', 1,inplace=True)
titanic_tickets_df['embarked'].replace('Q', 2,inplace=True) 
titanic_tickets_df['embarked'].astype(int)

titanic_tickets_df

,passenger_id,fare,embarked
0,1,7.2500,0
1,2,71.2833,1
2,3,7.9250,0
3,4,53.1000,0
4,5,8.0500,0
...,...,...,...
886,887,13.0000,0
887,888,30.0000,0
888,889,23.4500,0
889,890,30.0000,1


In [9]:
titanic_passengers_fg = fs.get_or_create_feature_group(
    name="titanic_passengers",
    version=1,
    primary_key=["passenger_id"]
)
titanic_passengers_fg.insert(titanic_passengers_df, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/398/fs/335/fg/8525


Uploading Dataframe: 0.00% |          | Rows 0/891 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/398/jobs/named/titanic_passengers_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fb3bbc0b220>, None)

In [10]:
titanic_tickets_fg = fs.get_or_create_feature_group(
    name="titanic_tickets",
    version=1,
    primary_key=["passenger_id"]
)
titanic_tickets_fg.insert(titanic_tickets_df, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/398/fs/335/fg/8526


Uploading Dataframe: 0.00% |          | Rows 0/891 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/398/jobs/named/titanic_tickets_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fb325f4c3a0>, None)

In [11]:
titanic_survived_fg = fs.get_or_create_feature_group(
    name="titanic_survived",
    version=1,
    primary_key=["passenger_id"]
)
titanic_survived_fg.insert(titanic_survived_df, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/398/fs/335/fg/8527


Uploading Dataframe: 0.00% |          | Rows 0/891 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/398/jobs/named/titanic_survived_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fb324356610>, None)